# Login to gcloud

In [ ]:
!gcloud auth login --no-launch-browser

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=1SlDJ01wxbdDBl9O_rJnLEFxD5YTAiJRuV21LE5f98Y&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account


Enter verification code: 4/1AFpsBm2HQ2h6DcgOWjN3n7raqsY0aGrNPuC4ing0r8aALadLLfBy4U

You are now logged in as [tzetterl@greenpeace.org].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


## Set gcloud project

In [ ]:
!gcloud config set project techlab-coding-team

Updated property [core/project].


# Install Libraries
In running environments this is created by requirements.txt

In [ ]:
!pip install tldextract
!pip install firebase-admin
!pip install beautifulsoup4
!pip install requests
!pip install pytz
!pip install html5lib
!pip install tldextract

## Create requirement.txt file

# Import all dependancies

In [ ]:
import logging
from urllib.parse import urlparse
from bs4 import BeautifulSoup as soup
from datetime import datetime, timedelta
import pytz
import re
import requests
import urllib.request
import socket
import tldextract
import json
import base64
import os

We are using firebase admin, and this section we are setting that up - you would need to have the gcp project name for your GCP project

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

#PROJECT_NAME = 'os.environ["PROJECT_NAME"]' when publishing cloud function
PROJECT_NAME = '<your project name>'

# initialize firebase sdk
CREDENTIALS = credentials.ApplicationDefault()
firebase_admin.initialize_app(CREDENTIALS, {
    'projectId': PROJECT_NAME,
})

# get firestore client
db = firestore.client()

Create an array that will hold the keywords that are store in firestore, the keywords are the words that will be used to search goole with.

Setting up we need to create an array were we will store the search queries, we also setup a few variables that will be used for createing a record for the database.

For the url request we are going to make we are defining a header and also how many search items should be returned.

In [ ]:
# Keyword Array filled in by daya in Firestore
keywords = []

def googlecloudbot_http(request):

    # Fake Real Browser
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'}

    results = 100 # valid options 10, 20, 30, 40, 50, and 100

    # Variables used
    ip = ''
    country = ''
    geolat = ''
    geolong = ''
    title = ''
    description = '' 

To get started we need to read the firestore database to get the search queries. We make a db call to a Firestore collection named searchquery.

We then start a for loop to read each document in the collection and only use the document were the column active is true.

In [ ]:
    searchquery_ref = db.collection(u'searchquery')
    
    for doc in searchquery_ref.where(u'active', u'==', True).stream():
        query = u'{}'.format(doc.to_dict()['queryterm'])

We are vuilding up the url we going to use to call the search providers website, in this case it's Google Search.

After that we start a while loop, so we can catch more than 100 search results, we go to the end of the url search.

We call the url with the headers.

In google you can get 429 which is that you exceeded the rate limit, if you get 429 you have to wait tills your allowed to continue. Using the Google search you don't want to run it every hour. It this case it's just runs once a week.

In [ ]:
        url = 'https://google.com/search?hl=en&q=' + query + '&source=lnms&sa=X' + '&num={}'.format(results)
    
        while url:
            page = requests.get(url, headers=headers)
            if page.status_code == 429:
                logging.info('Exceeded Rate Limit: {}')


After we receive a correct response 200, we will start parsing the data using the beautifulsoup python library.

We start a for loop to find the links the the html page that was rreturned using html elements to find the links we are intrested in.

In [ ]:
            searchsoup = soup(page.text, "html.parser")

            mydivs = searchsoup.find_all("div", class_="g") # for debugging purpose
    
            for link in searchsoup.find_all("a"):
                # get link details
                link_href = link.get('href')
                #Find the Title and Description

                if "url?q=" in link_href and not "webcache" in link_href:
                    # Try if Link is Active
                    try:
                        response = requests.get(link.get('href').split("?q=")[1].split("&sa=U")[0], timeout = 5)
                        active = True
                        logging.info("Link is active")
                    except:
                        active = False
                        logging.info("Link is inactive")


When we find a link, we unpack information from that link, one key element is the link text. We also cleaning up the link with a regex.

In [ ]:
                    title = link.text
                    title = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))', '', title, flags=re.MULTILINE)

The next we doing is to read a database collection named searchquerykeywords and we are focus on the documents in that collection that is active. This will be used to check if the title (link text) matches a word in the document. If match we continue with doing a duplicate check, see if the link already exixst. 

If we do not get match on the word in the title we ignore the link, the same goes for the the duplicate check, if we find a duplicate we ignore.

In [ ]:
               # Check if Keywords Exixst in Product title
                    searchkeywords_ref = db.collection(u'searchquerykeywords')        
                    # Request data from Firestore
                    for doc in searchkeywords_ref.where(u'active', u'==', True).stream():
                        keywords.append(u'{}'.format(doc.to_dict()['querykeywords']))
                                
                    if any(x in title for x in keywords):

                        # Duplicate check
                        docsurl = db.collection(u'searchlinks').where(u'url', u'==', link.get('href').split("?q=")[1].split("&sa=U")[0]).stream()
                        if (len(list(docsurl))):
                            logging.info("URL Exist, we will ignore")
                        else:
                            logging.info("URL Not found, we will add to databse")
                                

If the link(URL) was not found, we start building up the database document with the details we are intrested in. We also determine the hostname ip address of the link, and also try to get the geo data for that link. 

We also determine the name of the website by getting the name from the url, if the url is www.amazon.com - we get the name amazon from the url.

lastly we build the document that we store to the database.

In [ ]:
                           # Create a query against the collection
                            urllink = urlparse(link.get('href').split("?q=")[1].split("&sa=U")[0]).netloc
                            
                            # Find out what Country the IP address is from
                            hostname = socket.gethostname()
                            try:
                                ip = socket.gethostbyname(urllink)
                                try:
                                    with urllib.request.urlopen("https://geolocation-db.com/jsonp/" + ip) as geourl:
                                        geodata = geourl.read().decode()
                                        geodata = geodata.split("(")[1].strip(")")
                                        #print(geodata)
                                        geodata = json.loads(geodata)
                                        country = geodata["country_name"]
                                        geolat = geodata["latitude"]
                                        geolong = geodata["longitude"]
                                except:
                                    logging.info("Problem getting Country Details")
                            except socket.error:
                                logging.info("Socket Error IP could not be obtained")

                            # remove www/http or https from url
                            # Get the Shop name
                            shopurl = tldextract.extract(urllink)
                            shop = shopurl.domain

                            data = {
                                'title': title,
                                'description': description,
                                'shop': shop,
                                'date': _now(), # datetime object containing current date and time
                                'url': link.get('href').split("?q=")[1].split("&sa=U")[0],
                                'country': country,
                                'category': 'Google Search',
                                'search': query,
                                'ip_address': ip,
                                'status': active,
                                'lat': geolat,
                                'long': geolong
                            }
                            db.collection('searchlinks').document().set(data)  # Add a new doc in collection links with ID shop

The end of the code exit any for loop, or while loop is looking if there is another page and if so makes another request to get the next page.

When the Cloud Function completes - the cloud function goes to sleep.

In [ ]:
                    else:
                        logging.info("No match on Keywords")    # -> <match object>
            # get next page url
            url = searchsoup.find('a', id='pnnext')
            if url:
                url = 'https://www.google.com/' + url['href'] + '&source=lnms&sa=X'
            else:
                logging.info('Search Completed: {}')
    #return "All Done"

We have a sub function defined to set the date

In [ ]:
def _now():
    return datetime.utcnow().replace(tzinfo=pytz.utc).strftime('%Y-%m-%d %H:%M:%S %Z')

Another sub function is used to clean a url, to remove anything in the url after defined charachters so we do not include utm parameters or other marketing or tracking codes in a url

In [ ]:
def cleanurl(url):
    matches = re.findall('(.+\?)([^#]*)(.*)', url)
    if len(matches) == 0:
        return url
    match = matches[0]
    query = match[1]
    return match[0]


We convert the urls

In [ ]:
def convert(url):
    if url.startswith('http://www.'):
        return 'http://' + url[len('http://www.'):]
    if url.startswith('//www.'):
        return 'https://www' + url[len('//www'):]
    if url.startswith('//image.'):
        return 'https://' + url[len('//'):]
    if url.startswith('www.'):
        return 'https://' + url[len('www.'):]
    if not url.startswith('http://'):
        return 'http://' + url
    return url

# Cloud Function

In this section I'm showing you how you can deploy your cloud function from a a Colab Notebook.

The first part here we are creatating the requirements.txt file

In [ ]:
%%writefile requirements.txt

firebase-admin==2.11.0
beautifulsoup4==4.8.2
requests==2.22.0
pytz==2019.3
html5lib==1.0.1
tldextract==2.2.2

Writing requirements.txt


Here we are creating the main.py file, to make this work on your project you need to change the project name in your Cloud Function after deployment. You do that by adding a OS variable to your runtime.
You can also do it through the gcloud command.

In [ ]:
%%writefile main.py

import logging
from urllib.parse import urlparse
from bs4 import BeautifulSoup as soup
from datetime import datetime, timedelta
import pytz
import re
import requests
import urllib.request
import socket
import tldextract
import json
import os

import base64
import os

import firebase_admin
from firebase_admin import credentials, firestore

PROJECT_NAME = os.environ["PROJECT_NAME"]

# initialize firebase sdk
CREDENTIALS = credentials.ApplicationDefault()
firebase_admin.initialize_app(CREDENTIALS, {
    'projectId': PROJECT_NAME,
})

# get firestore client
db = firestore.client()

# Keyword Array filled in by daya in Firestore
keywords = []

def googlecloudbot_http(request):

    # Fake Real Browser
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'}

    results = 100 # valid options 10, 20, 30, 40, 50, and 100

    # Variables used
    ip = ''
    country = ''
    geolat = ''
    geolong = ''
    title = ''
    description = '' 

    # Get available proxies
    #cworking_proxies = check_proxies()

    searchquery_ref = db.collection(u'searchquery')
#    for prox in list(working_proxies):
    
    for doc in searchquery_ref.where(u'active', u'==', True).stream():
        query = u'{}'.format(doc.to_dict()['queryterm'])
        print(query)
        url = 'https://google.com/search?hl=en&q=' + query + '&source=lnms&sa=X' + '&num={}'.format(results)
    
        while url:
            print(url)

            page = requests.get(url, headers=headers)
            if page.status_code == 429:
                logging.info('Exceeded Rate Limit: {}')

            searchsoup = soup(page.text, "html.parser")
            # searchsoup = soup(page.text, "html5lib")

            mydivs = searchsoup.find_all("div", class_="g") # for debugging purpose
            # print(searchsoup.get_text()) # for debugging purpose   
    
            for link in searchsoup.find_all("a"):
                # get link details
                link_href = link.get('href')
                #Find the Title and Description

                if "url?q=" in link_href and not "webcache" in link_href:
                    # Try if Link is Active
                    try:
                        response = requests.get(link.get('href').split("?q=")[1].split("&sa=U")[0], timeout = 5)
                        active = True
                        logging.info("Link is active")
                    except:
                        active = False
                        logging.info("Link is inactive")

                    title = link.text
                    title = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))', '', title, flags=re.MULTILINE)
                    # Check if Keywords Exixst in Product title
                    searchkeywords_ref = db.collection(u'searchquerykeywords')        
                    # Request data from Firestore
                    for doc in searchkeywords_ref.where(u'active', u'==', True).stream():
                        keywords.append(u'{}'.format(doc.to_dict()['querykeywords']))
                                
                    if any(x in title for x in keywords):

                        # Duplicate check
                        docsurl = db.collection(u'searchlinks').where(u'url', u'==', link.get('href').split("?q=")[1].split("&sa=U")[0]).stream()
                        if (len(list(docsurl))):
                            logging.info("URL Exist, we will ignore")
                        else:
                            logging.info("URL Not found, we will add to databse")
                                
                            # Create a query against the collection
                            urllink = urlparse(link.get('href').split("?q=")[1].split("&sa=U")[0]).netloc
                            
                            # Find out what Country the IP address is from
                            hostname = socket.gethostname()
                            try:
                                ip = socket.gethostbyname(urllink)
                                try:
                                    with urllib.request.urlopen("https://geolocation-db.com/jsonp/" + ip) as geourl:
                                        geodata = geourl.read().decode()
                                        geodata = geodata.split("(")[1].strip(")")
                                        #print(geodata)
                                        geodata = json.loads(geodata)
                                        country = geodata["country_name"]
                                        geolat = geodata["latitude"]
                                        geolong = geodata["longitude"]
                                except:
                                    logging.info("Problem getting Country Details")
                            except socket.error:
                                logging.info("Socket Error IP could not be obtained")

                            # remove www/http or https from url
                            # Get the Shop name
                            shopurl = tldextract.extract(urllink)
                            shop = shopurl.domain

                            data = {
                                'title': title,
                                'description': description,
                                'shop': shop,
                                'date': _now(), # datetime object containing current date and time
                                'url': link.get('href').split("?q=")[1].split("&sa=U")[0],
                                'country': country,
                                'category': 'Google Search',
                                'search': query,
                                'ip_address': ip,
                                'status': active,
                                'lat': geolat,
                                'long': geolong
                            }
                            db.collection('searchlinks').document().set(data)  # Add a new doc in collection links with ID shop
                    else:
                        logging.info("No match on Keywords")    # -> <match object>
            # get next page url
            url = searchsoup.find('a', id='pnnext')
            if url:
                url = 'https://www.google.com/' + url['href'] + '&source=lnms&sa=X'
            else:
                logging.info('Search Completed: {}')
    #return "All Done"

    # Send a message
   # _sendmessage()
def _now():
    return datetime.utcnow().replace(tzinfo=pytz.utc).strftime('%Y-%m-%d %H:%M:%S %Z')

def cleanurl(url):
    matches = re.findall('(.+\?)([^#]*)(.*)', url)
    if len(matches) == 0:
        return url
    match = matches[0]
    query = match[1]
    return match[0]

def convert(url):
    if url.startswith('http://www.'):
        return 'http://' + url[len('http://www.'):]
    if url.startswith('//www.'):
        return 'https://www' + url[len('//www'):]
    if url.startswith('//image.'):
        return 'https://' + url[len('//'):]
    if url.startswith('www.'):
        return 'https://' + url[len('www.'):]
    if not url.startswith('http://'):
        return 'http://' + url
    return url

googlecloudbot_http('request')

## Deploy Cloud Function

In [ ]:
!gcloud functions deploy googlecloudbot_http --region=europe-west1 --memory=256MB --timeout 256 --runtime python310 --set-env-vars PROJECT_NAME=<gcp_project_name> --trigger-http --allow-unauthenticated 